# Identifying Similar Images with Tensorflow

Douglas Duhaime wrote an [excellent tutorial](http://douglasduhaime.com/posts/identifying-similar-images-with-tensorflow.html). This notebook is my experiment trying to follow his steps, using a github repo and a jupyter binder.

I think it works...

ok, so: I have a bunch of images in \images (we're pulling from [this repo](https://github.com/shawngraham/bindr-test-Identifying-Similar-Images-with-TensorFlow)). In Duhaime's tutorial, he had a folder with 2000 images. I'm just going with 25 here because a) I'm impatient and b) I don't know how many I can push into github.

The code below all runs, but I'm not sure where the output is hiding...

So. Install the needful things, run the modified classify script (which pulls out the second but last layer), and write it to the new `image_vectors` dir. Then cluster, then project.

In [1]:
!pip install psutil

In [2]:
!pip install numpy

In [3]:
!pip install tensorflow

In [4]:
!pip install annoy

In [5]:
!pip install scipy

In [6]:
!pip install nltk

In [7]:
!pip install sklearn

ok. that's the last of the stuff you need to install.

---

## Now Let's Identify Similar Images

In [8]:
!python classify_images.py "images/*"

>> Downloading inception-2015-12-05.tgz 100.0%
Succesfully downloaded inception-2015-12-05.tgz 88931400 bytes.
2018-05-07 09:52:20.550199: W tensorflow/core/framework/op_def_util.cc:346] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
2018-05-07 09:52:20.802865: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
parsing 0 images/3969575.jpg 

results for images/3969575.jpg
tobacco shop, tobacconist shop, tobacconist (score = 0.27585)


results for images/3969575.jpg
bookshop, bookstore, bookstall (score = 0.19276)


results for images/3969575.jpg
entertainment center (score = 0.05562)


results for images/3969575.jpg
bookcase (score = 0.04556)


results for images/3969575.jpg
lab coat, laboratory coat (score = 0.03591)


parsing 1 images/3969603.jpg 

results for images/3969603.jpg
book jacket, dust cover, dust j

results for images/3969609.jpg
stretcher (score = 0.25366)


results for images/3969609.jpg
apiary, bee house (score = 0.19872)


results for images/3969609.jpg
mountain tent (score = 0.03312)


results for images/3969609.jpg
thresher, thrasher, threshing machine (score = 0.02733)


results for images/3969609.jpg
park bench (score = 0.02342)


parsing 20 images/3969591.jpg 

results for images/3969591.jpg
ox (score = 0.86663)


results for images/3969591.jpg
oxcart (score = 0.02592)


results for images/3969591.jpg
whippet (score = 0.00860)


results for images/3969591.jpg
plow, plough (score = 0.00395)


results for images/3969591.jpg
worm fence, snake fence, snake-rail fence, Virginia fence (score = 0.00303)


parsing 21 images/3969585.jpg 

results for images/3969585.jpg
guillotine (score = 0.12392)


results for images/3969585.jpg
lumbermill, sawmill (score = 0.10771)


results for images/3969585.jpg
prison, prison house (score = 0.07205)


results for images/3969585.jpg
butcher sh

now cluster with his nearest neighbours script. "Each of those outfiles will identify the 30 images most similar to the given image. To search for more or fewer nearest neighbors, one just needs to update the n_nearest_neighbors variable in the nearest neighbors script."

In [9]:
!python cluster_vectors.py

and now we run the t-sne clustering script

In [11]:
# ok let's project this stuff!
from sklearn.manifold import TSNE
import numpy as np
import glob, json, os

# create datastores
vector_files = []
image_vectors = []
chart_data = []
# note the maximum here - change up for total number you've got!
maximum_imgs = 25

# build a list of image vectors
vector_files = glob.glob('image_vectors/*.npz')[:maximum_imgs]
for c, i in enumerate(vector_files):
  image_vectors.append(np.loadtxt(i))
  print(' * loaded', c, 'of', len(vector_files), 'image vectors')

# build the tsne model on the image vectors
print('building tsne model')
model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
fit_model = model.fit_transform( np.array(image_vectors) )
 
# store the coordinates of each image in the chart data
for c, i in enumerate(fit_model):
  image_name = os.path.basename(vector_files[c]).replace('.npz', '') 
  chart_data.append({
    'image': os.path.join('images', image_name),
    'x': i[0],
    'y': i[1]
  })

 * loaded 0 of 25 image vectors
 * loaded 1 of 25 image vectors
 * loaded 2 of 25 image vectors
 * loaded 3 of 25 image vectors
 * loaded 4 of 25 image vectors
 * loaded 5 of 25 image vectors
 * loaded 6 of 25 image vectors
 * loaded 7 of 25 image vectors
 * loaded 8 of 25 image vectors
 * loaded 9 of 25 image vectors
 * loaded 10 of 25 image vectors
 * loaded 11 of 25 image vectors
 * loaded 12 of 25 image vectors
 * loaded 13 of 25 image vectors
 * loaded 14 of 25 image vectors
 * loaded 15 of 25 image vectors
 * loaded 16 of 25 image vectors
 * loaded 17 of 25 image vectors
 * loaded 18 of 25 image vectors
 * loaded 19 of 25 image vectors
 * loaded 20 of 25 image vectors
 * loaded 21 of 25 image vectors
 * loaded 22 of 25 image vectors
 * loaded 23 of 25 image vectors
 * loaded 24 of 25 image vectors
building tsne model


then we write the `chart_data` to file.

In [12]:
thefile = open('image_tsne_projections.json', 'w')
for item in chart_data:
  thefile.write("%s\n" % item)
thefile.close()

Now go view your work by clicking on the 'jupyter' button above! Outputs, folders, etc, all visible there.

Before you can work with that json file though, you need to add a `[` and a `]` to the front and end of the file, and a `,` at the end of each line (except the last one). Easily done in Regex; doing it in the code chunk is a bit beyond me still.